In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
#from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, num_X2feature)))
    y_ = torch.mm(X2_before_,true_w2)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.mm(X2_before_,true_w2))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [8]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(100,101,100): 
        for num_X2feature in [20,2,5,10]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1)=generatedata(noise=0.3)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.003
                    results_S=[]
                    t=0
                    before1=0
                    while True:                     
                        Y1=y
                        Y2=torch.mm(X2,w2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 

                        #print('Loss',t,'=',Loss)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-8:
                            break
                        before1=Loss
                        if t>=200:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    results_error.append(error_each)
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')

                #print('真实置换矩阵为：',true_P)
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print(num_X2feature,num_example,'置换矩阵误差：',error_P,end='   ')
                error_reg2=(torch.norm(y_-torch.mm(torch.mm(S,X2_),w2))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

###################################################################################### 20 100 平均相对误差2： tensor(4.3334, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0662, dtype=torch.float64)   实验回归误差 tensor(0.0882, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################################################################### 20 100 平均相对误差2： tensor(6.4087, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4071, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0577, dtype=torch.float64)   实验回归误差 tensor(0.0748, dtype=torch.float64, grad_fn=<DivBackward0>)
############################################################################################# 20 100 平均相对误差2： tensor(4.8408, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4071, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0785, dtype

################################################################################################### 5 100 平均相对误差2： tensor(0.4140, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.3491, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1868, dtype=torch.float64)   实验回归误差 tensor(0.0876, dtype=torch.float64, grad_fn=<DivBackward0>)
################################### 5 100 平均相对误差2： tensor(2.1668, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.4071, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1937, dtype=torch.float64)   实验回归误差 tensor(0.1063, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################################################################ 5 100 平均相对误差2： tensor(3.2153, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.4071, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0769, dtype=torch.float64)   实验回归误差 tensor(0.1417, dtype=torch.fl

################################################################################################################ 20 100 平均相对误差2： tensor(6.6732, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0601, dtype=torch.float64)   实验回归误差 tensor(0.0508, dtype=torch.float64, grad_fn=<DivBackward0>)
###################################################################################################################### 20 100 平均相对误差2： tensor(4.8577, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0843, dtype=torch.float64)   实验回归误差 tensor(0.0725, dtype=torch.float64, grad_fn=<DivBackward0>)
########################### 2 100 平均相对误差2： tensor(0.0618, dtype=torch.float64, grad_fn=<NormBackward0>)   2 100 置换矩阵误差： tensor(1.3038, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1923, dtype=torch.float64)   实验回归误

############################################################################################################################################## 10 100 平均相对误差2： tensor(3.5671, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1325, dtype=torch.float64)   实验回归误差 tensor(0.0784, dtype=torch.float64, grad_fn=<DivBackward0>)
######################################## 10 100 平均相对误差2： tensor(4.0700, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0998, dtype=torch.float64)   实验回归误差 tensor(0.1596, dtype=torch.float64, grad_fn=<DivBackward0>)
############################################################################################################################ 10 100 平均相对误差2： tensor(5.9280, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)  

############################################################################################ 5 100 平均相对误差2： tensor(3.2487, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1285, dtype=torch.float64)   实验回归误差 tensor(0.2072, dtype=torch.float64, grad_fn=<DivBackward0>)
################################## 5 100 平均相对误差2： tensor(1.9026, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.2147, dtype=torch.float64)   实验回归误差 tensor(0.1235, dtype=torch.float64, grad_fn=<DivBackward0>)
########################################################################## 5 100 平均相对误差2： tensor(1.7373, dtype=torch.float64, grad_fn=<NormBackward0>)   5 100 置换矩阵误差： tensor(1.3856, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1258, dtype=torch.float64)   实验回归误差 tensor(0.1091, dtype=torch.float64, grad_fn=<DivBackward0>)

############################################################################################################################################################################ 20 100 平均相对误差2： tensor(5.4648, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0975, dtype=torch.float64)   实验回归误差 tensor(0.0708, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################################################################################################ 20 100 平均相对误差2： tensor(4.3045, dtype=torch.float64, grad_fn=<NormBackward0>)   20 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0704, dtype=torch.float64)   实验回归误差 tensor(0.0437, dtype=torch.float64, grad_fn=<DivBackward0>)
############################################################################# 20 100 平均相对误差2： tensor(4.6239, dtype=torch.float64, grad_fn=<NormBackward0>)   20 10

################################################################################# 10 100 平均相对误差2： tensor(2.4096, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.3928, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1190, dtype=torch.float64)   实验回归误差 tensor(0.1024, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################################################################################# 10 100 平均相对误差2： tensor(4.0143, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0806, dtype=torch.float64)   实验回归误差 tensor(0.0901, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################### 10 100 平均相对误差2： tensor(2.6855, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.3843, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1278, dtype=torch.float64)   实验回归误差 tensor(0

###################################### 2 100 平均相对误差2： tensor(0.2047, dtype=torch.float64, grad_fn=<NormBackward0>)   2 100 置换矩阵误差： tensor(1.3416, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.2866, dtype=torch.float64)   实验回归误差 tensor(0.0927, dtype=torch.float64, grad_fn=<DivBackward0>)
########################### 2 100 平均相对误差2： tensor(0.0976, dtype=torch.float64, grad_fn=<NormBackward0>)   2 100 置换矩阵误差： tensor(1.3565, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.2461, dtype=torch.float64)   实验回归误差 tensor(0.0789, dtype=torch.float64, grad_fn=<DivBackward0>)
###################### 2 100 平均相对误差2： tensor(1.1065, dtype=torch.float64, grad_fn=<NormBackward0>)   2 100 置换矩阵误差： tensor(1.3928, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.2565, dtype=torch.float64)   实验回归误差 tensor(0.1440, dtype=torch.float64, grad_fn=<DivBackward0>)
################################################################# 2 100 平均相对误差2： tensor(0.1983, dtype=torch.floa

################################################################################################################################# 10 100 平均相对误差2： tensor(6.1155, dtype=torch.float64, grad_fn=<NormBackward0>)   10 100 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0776, dtype=torch.float64)   实验回归误差 tensor(0.0979, dtype=torch.float64, grad_fn=<DivBackward0>)
